# 先分析性能，然后再优化

In [1]:
from sys import stdout as STDOUT

**示例：**想查明程序中的某个算法为什么运行得比较慢。下面定义的这个函数，采用插入排序法来排列一组数据。

In [2]:
def insertion_sort(data):
    result = []
    for value in data:
        insert_value(result, value)
    return result

下面定义的函数，要对外界输入的array序列进行线性扫描，以确定插入点。因此，它的效率是非常低的

In [3]:
def insert_value(array, value):
    for i, existing in enumerate(array):
        if existing > value:
            array.insert(i, value)
            return
    array.append(value)

In [4]:
from random import randint

max_size = 10**4
data = [randint(0, max_size) for _ in range(max_size)]
test = lambda: insertion_sort(data)

采用内置的cProfile模块，因为它在做性能分析时，对受测程序的效率只会产生很小的影响。

In [5]:
from cProfile import Profile

profiler = Profile()
profiler.runcall(test)

[0,
 2,
 3,
 4,
 8,
 8,
 9,
 9,
 9,
 10,
 11,
 12,
 13,
 15,
 15,
 16,
 16,
 20,
 21,
 23,
 25,
 25,
 27,
 29,
 30,
 32,
 32,
 33,
 34,
 34,
 35,
 36,
 39,
 40,
 40,
 41,
 45,
 46,
 47,
 48,
 50,
 50,
 52,
 55,
 55,
 55,
 57,
 58,
 59,
 59,
 61,
 64,
 65,
 66,
 66,
 70,
 70,
 71,
 72,
 73,
 73,
 74,
 75,
 80,
 80,
 80,
 82,
 83,
 83,
 84,
 84,
 84,
 85,
 85,
 86,
 86,
 87,
 91,
 92,
 94,
 95,
 96,
 96,
 98,
 99,
 100,
 104,
 104,
 106,
 107,
 108,
 109,
 109,
 109,
 110,
 111,
 113,
 114,
 115,
 117,
 118,
 118,
 121,
 122,
 123,
 126,
 128,
 130,
 131,
 133,
 134,
 134,
 136,
 137,
 137,
 138,
 139,
 139,
 139,
 140,
 141,
 142,
 142,
 143,
 143,
 145,
 146,
 146,
 147,
 148,
 149,
 149,
 151,
 151,
 153,
 154,
 156,
 156,
 158,
 158,
 158,
 159,
 159,
 161,
 161,
 161,
 163,
 164,
 166,
 166,
 167,
 168,
 170,
 170,
 171,
 171,
 173,
 175,
 178,
 179,
 180,
 181,
 181,
 181,
 181,
 182,
 183,
 185,
 185,
 185,
 187,
 187,
 187,
 188,
 189,
 189,
 189,
 192,
 195,
 195,
 196,
 197,
 1

test函数运行完毕之后，采用内置的pstats模块和模块中的Stats类，来剖析由Profile对象所手机到的性能统计数据。

In [6]:
import sys
from pstats import Stats

stats = Stats(profiler)
stats = Stats(profiler, stream=STDOUT)
stats.strip_dirs()
stats.sort_stats('cumulative')
stats.print_stats()

         20003 function calls in 4.336 seconds

   Ordered by: cumulative time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.000    0.000    4.336    4.336 <ipython-input-4-1054ed032f49>:5(<lambda>)
        1    0.007    0.007    4.336    4.336 <ipython-input-2-cbe174de868f>:1(insertion_sort)
    10000    4.290    0.000    4.328    0.000 <ipython-input-3-c4fc1ad100fb>:1(insert_value)
     9991    0.038    0.000    0.038    0.000 {method 'insert' of 'list' objects}
        9    0.000    0.000    0.000    0.000 {method 'append' of 'list' objects}
        1    0.000    0.000    0.000    0.000 {method 'disable' of '_lsprof.Profiler' objects}




**改进：**改用内置的bisect模块来重新定义此函数

In [7]:
from bisect import bisect_left

def insert_value(array, value):
    i = bisect_left(array, value)
    array.insert(i, value)

In [8]:
profiler = Profile()
profiler.runcall(test)
stats = Stats(profiler, stream=STDOUT)
stats.strip_dirs()
stats.sort_stats('cumulative')
stats.print_stats()

         30003 function calls in 0.065 seconds

   Ordered by: cumulative time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.000    0.000    0.065    0.065 <ipython-input-4-1054ed032f49>:5(<lambda>)
        1    0.005    0.005    0.065    0.065 <ipython-input-2-cbe174de868f>:1(insertion_sort)
    10000    0.009    0.000    0.060    0.000 <ipython-input-7-9e3ac1f6182f>:3(insert_value)
    10000    0.035    0.000    0.035    0.000 {method 'insert' of 'list' objects}
    10000    0.016    0.000    0.016    0.000 {built-in method _bisect.bisect_left}
        1    0.000    0.000    0.000    0.000 {method 'disable' of '_lsprof.Profiler' objects}




**Profile分析缺陷：**在分析整个程序的性能时，可能会发现，某个常用的工具函数，占据了大部分执行时间。但从profiler所给出的默认统计数据中，我们却无法清晰地分辨出，程序中的不同部分，究竟是如何调用这个工具函数的。

**示例：**程序中有两个不同的函数，都会频繁调用这个名为my_utility的工具函数。

In [9]:
def my_utility(a, b):
    c = 1
    for i in range(100):
        c += a * b

def first_func():
    for _ in range(1000):
        my_utility(4, 5)

def second_func():
    for _ in range(10):
        my_utility(1, 3)

def my_program():
    for _ in range(20):
        first_func()
        second_func()

In [10]:
profiler = Profile()
profiler.runcall(my_program)
stats = Stats(profiler, stream=STDOUT)
stats.strip_dirs()
stats.sort_stats('cumulative')
stats.print_stats()

         20242 function calls in 0.400 seconds

   Ordered by: cumulative time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.000    0.000    0.400    0.400 <ipython-input-9-224d37765872>:14(my_program)
       20    0.009    0.000    0.397    0.020 <ipython-input-9-224d37765872>:6(first_func)
    20200    0.391    0.000    0.391    0.000 <ipython-input-9-224d37765872>:1(my_utility)
       20    0.000    0.000    0.003    0.000 <ipython-input-9-224d37765872>:10(second_func)
        1    0.000    0.000    0.000    0.000 {method 'disable' of '_lsprof.Profiler' objects}




**缺陷解决办法：**可以调用stats.print_callers()方法，可以在性能分析数据中列出每个函数的调用者，可以据此看出该函数所耗费的执行时间，究竟是由哪些调用者所分别引起的。

In [11]:
stats = Stats(profiler, stream=STDOUT)
stats.strip_dirs()
stats.sort_stats('cumulative')
stats.print_callers()

   Ordered by: cumulative time

Function                                          was called by...
                                                      ncalls  tottime  cumtime
<ipython-input-9-224d37765872>:14(my_program)     <- 
<ipython-input-9-224d37765872>:6(first_func)      <-      20    0.009    0.397  <ipython-input-9-224d37765872>:14(my_program)
<ipython-input-9-224d37765872>:1(my_utility)      <-   20000    0.388    0.388  <ipython-input-9-224d37765872>:6(first_func)
                                                         200    0.003    0.003  <ipython-input-9-224d37765872>:10(second_func)
<ipython-input-9-224d37765872>:10(second_func)    <-      20    0.000    0.003  <ipython-input-9-224d37765872>:14(my_program)
{method 'disable' of '_lsprof.Profiler' objects}  <- 


